In [ ]:
# Step 1: Model Owner Uploads Model Architecture and Weights

In [ ]:
# Step 2: Model Owners Sets up Endpoint for inference

In [1]:
# third party
import torch

# syft absolute
import syft as sy

In [ ]:
domain_client = None


@sy.api_endpoint_method()
def public_inference(context, prompt: torch.Tensor) -> torch.Tensor:
    return torch.tensor([1, 2, 3])


@sy.api_endpoint_method()
def private_inference(context, prompt: torch.Tensor) -> torch.Tensor:
    # stdlib
    from pathlib import Path

    # third party
    from swin_zoo.model_arch import SimpleModel
    import torch

    prompt = torch.tensor([[4.0]])
    admin_client = context.node.get_guest_client()
    admin_client.credentials = context.node.signing_key
    model = admin_client.datasets[0].assets[0]

    res = model.write_folder_to_current_path()
    assert res

    model_folder_path = Path("./swin_zoo")
    model = SimpleModel()
    model.load_state_dict(torch.load(f"{model_folder_path}/model_weights.pt"))
    model.eval()

    output = model(prompt)
    output = output.detach().numpy()

    return output

In [ ]:
new_endpoint = sy.TwinAPIEndpoint(
    path="model.inference",
    mock_function=public_inference,
    private_function=private_inference,
    description="Lore ipsulum ...",
)

In [ ]:
response = domain_client.api.services.api.delete(endpoint_path="model.inference")
response

In [ ]:
response = domain_client.api.services.api.add(endpoint=new_endpoint)
response

In [ ]:
domain_client.api.services.model.inference(prompt=torch.Tensor([4.0]))

In [ ]:
# Step 3: DS creates function based on endpoint
ds_client = None 
prompt = ds_client.api.services.action.set(torch.tensor([1]))

@sy.syft_function_single_use(
    endpoint=domain_client.api.services.model.inference,
    input_prompt = prompt.id
    worker_pool_name=worker_pool_name,
)
def job_function(endpoint, input_prompt):

    # Do some pre-processing steps

    result = endpoint(
        prompt = input_prompt
    )

    # Do some post-processing steps
    return result

In [ ]:
# Step 4: DS Submits Projects

In [ ]:
# Step 5: DO approves project